# Split into small Parquet files

In [ ]:
sc.stop()

In [1]:
%%time
import sys
sys.path.append('./lib')
from numpy_pack import packArray,unpackArray
from binary_search import binary_search

from pyspark import SparkContext
sc = SparkContext(pyFiles=['lib/numpy_pack.py'])

CPU times: user 248 ms, sys: 16 ms, total: 264 ms
Wall time: 13.4 s


In [2]:
Conf=sc.getConf()
#Conf.getAll()

In [42]:
!aws s3 ls dse-weather

                           PRE 256_STAT/
                           PRE NY.parquet/
                           PRE US_Weather_with_smoothed.parquet/
                           PRE US_stations.parquet/
                           PRE US_weather.parquet/
                           PRE info/
                           PRE weather.parquet/
2016-02-09 04:09:49 1511481989 ALL.csv.gz
2018-04-18 15:48:12        715 ALLBootstrap.sh
2018-04-18 15:48:13        539 MasterBootstrap.sh
2018-04-17 02:25:22        600 PrivateBootstrap.sh
2018-04-18 15:48:14        616 RunFromTerminal.sh
2018-04-19 00:06:22          0 US_Weather_with_smoothed.parquet_$folder$
2018-04-18 15:48:11        421 s3hook.sh


In [78]:
# Open public bucket (readable by anybody with an AWS account, but not through a ublic URL)
bucket='dse-weather'
dir='/home/ec2-user/spark/'
s3helper.open_bucket(bucket)
from time import time

In [79]:
src= "weather.parquet/"
files = s3helper.bucket.list(prefix=src)
i=0
for file in files:
    print(i,file.key)
    i+=1

0 weather.parquet/_SUCCESS
1 weather.parquet/part-00000-6cb19187-62a0-42ad-9516-e03e05ea0c40-c000.snappy.parquet
2 weather.parquet/part-00001-6cb19187-62a0-42ad-9516-e03e05ea0c40-c000.snappy.parquet
3 weather.parquet/part-00002-6cb19187-62a0-42ad-9516-e03e05ea0c40-c000.snappy.parquet
4 weather.parquet/part-00003-6cb19187-62a0-42ad-9516-e03e05ea0c40-c000.snappy.parquet
5 weather.parquet/part-00004-6cb19187-62a0-42ad-9516-e03e05ea0c40-c000.snappy.parquet
6 weather.parquet/part-00005-6cb19187-62a0-42ad-9516-e03e05ea0c40-c000.snappy.parquet
7 weather.parquet/part-00006-6cb19187-62a0-42ad-9516-e03e05ea0c40-c000.snappy.parquet
8 weather.parquet/part-00007-6cb19187-62a0-42ad-9516-e03e05ea0c40-c000.snappy.parquet
9 weather.parquet/part-00008-6cb19187-62a0-42ad-9516-e03e05ea0c40-c000.snappy.parquet
10 weather.parquet/part-00009-6cb19187-62a0-42ad-9516-e03e05ea0c40-c000.snappy.parquet
11 weather.parquet/part-00010-6cb19187-62a0-42ad-9516-e03e05ea0c40-c000.snappy.parquet
12 weather.parquet/part-0

In [80]:
s3helper.conn

S3Connection:s3.amazonaws.com

In [81]:
file_names=['weather.parquet/']

In [82]:
t=time()
for file in file_names:
    t=time()
    !hadoop fs -rm -r /$file
    print('deleted %s in %4.2f seconds'%(file, time()-t))
    t=time()
    tmp = s3helper.s3_to_hdfs('%s'%file, '/%s'%file)
    print('Downloading %s in %4.2f seconds'%(file, time()-t))
    !~/hadoop/bin/hadoop fs -ls -h /$file
## copying took 62.695912838, 74.9509780407, 78.921860218 seconds

rm: `/weather.parquet/': No such file or directory
deleted weather.parquet/ in 2.14 seconds
s3_to_hdfs: src= "weather.parquet/", tgt="/weather.parquet/"
s3_to_hdfs, self.bucket.list(prefix=src), src= weather.parquet/
s3_to_hdfs, src= weather.parquet/
files=[s3n://dse-weather/weather.parquet/_SUCCESS s3n://dse-weather/weather.parquet/part-00000-6cb19187-62a0-42ad-9516-e03e05ea0c40-c000.snappy.parquet s3n://dse-weather/weather.parquet/part-00001-6cb19187-62a0-42ad-9516-e03e05ea0c40-c000.snappy.parquet s3n://dse-weather/weather.parquet/part-00002-6cb19187-62a0-42ad-9516-e03e05ea0c40-c000.snappy.parquet s3n://dse-weather/weather.parquet/part-00003-6cb19187-62a0-42ad-9516-e03e05ea0c40-c000.snappy.parquet s3n://dse-weather/weather.parquet/part-00004-6cb19187-62a0-42ad-9516-e03e05ea0c40-c000.snappy.parquet s3n://dse-weather/weather.parquet/part-00005-6cb19187-62a0-42ad-9516-e03e05ea0c40-c000.snappy.parquet s3n://dse-weather/weather.parquet/part-00006-6cb19187-62a0-42ad-9516-e03e05ea0c40-c000.

about to run  command:'/usr/bin/hdfs dfs -cp s3n://dse-weather/weather.parquet/_SUCCESS s3n://dse-weather/weather.parquet/part-00000-6cb19187-62a0-42ad-9516-e03e05ea0c40-c000.snappy.parquet s3n://dse-weather/weather.parquet/part-00001-6cb19187-62a0-42ad-9516-e03e05ea0c40-c000.snappy.parquet s3n://dse-weather/weather.parquet/part-00002-6cb19187-62a0-42ad-9516-e03e05ea0c40-c000.snappy.parquet s3n://dse-weather/weather.parquet/part-00003-6cb19187-62a0-42ad-9516-e03e05ea0c40-c000.snappy.parquet s3n://dse-weather/weather.parquet/part-00004-6cb19187-62a0-42ad-9516-e03e05ea0c40-c000.snappy.parquet s3n://dse-weather/weather.parquet/part-00005-6cb19187-62a0-42ad-9516-e03e05ea0c40-c000.snappy.parquet s3n://dse-weather/weather.parquet/part-00006-6cb19187-62a0-42ad-9516-e03e05ea0c40-c000.snappy.parquet s3n://dse-weather/weather.parquet/part-00007-6cb19187-62a0-42ad-9516-e03e05ea0c40-c000.snappy.parquet s3n://dse-weather/weather.parquet/part-00008-6cb19187-62a0-42ad-9516-e03e05ea0c40-c000.snappy.pa

b'Found 59 items\n'
b'-rw-r--r--   3 hadoop hadoop          0 2020-04-26 21:35 /weather.parquet/_SUCCESS\n'
b'-rw-r--r--   3 hadoop hadoop   40670401 2020-04-26 21:35 /weather.parquet/part-00000-6cb19187-62a0-42ad-9516-e03e05ea0c40-c000.snappy.parquet\n'
b'-rw-r--r--   3 hadoop hadoop   40307528 2020-04-26 21:35 /weather.parquet/part-00001-6cb19187-62a0-42ad-9516-e03e05ea0c40-c000.snappy.parquet\n'
b'-rw-r--r--   3 hadoop hadoop   40016618 2020-04-26 21:35 /weather.parquet/part-00002-6cb19187-62a0-42ad-9516-e03e05ea0c40-c000.snappy.parquet\n'
b'-rw-r--r--   3 hadoop hadoop   40377232 2020-04-26 21:35 /weather.parquet/part-00003-6cb19187-62a0-42ad-9516-e03e05ea0c40-c000.snappy.parquet\n'
b'-rw-r--r--   3 hadoop hadoop   40119938 2020-04-26 21:35 /weather.parquet/part-00004-6cb19187-62a0-42ad-9516-e03e05ea0c40-c000.snappy.parquet\n'
b'-rw-r--r--   3 hadoop hadoop   40278884 2020-04-26 21:35 /weather.parquet/part-00005-6cb19187-62a0-42ad-9516-e03e05ea0c40-c000.snappy.parquet\n'
b'-rw-r--r

/bin/sh: /home/hadoop/hadoop/bin/hadoop: No such file or directory


In [86]:
%%writefile ~/s3helper2.py
from boto.exception import S3ResponseError
from boto.s3.connection import S3Connection
import os
import subprocess


def _run_command(command, detail=True):
    if detail:
        print("about to run  command:'%s'"%command)
    proc = subprocess.Popen(command.split(),
                            stdout=subprocess.PIPE,
                            stderr=subprocess.PIPE)
    if detail:
        while True:
            line = proc.stdout.readline()
            if not line:
                break
            print(line),
    return proc.communicate()


def _list_hdfs(path):
    command = "/usr/bin/hdfs dfs -ls %s" % path
    out, err = _run_command(command)
    if out:
        print(out)


def _s3_to_hdfs(files, tgt):
    print("_s3_to_hdfs files=%s, tgt=%s"%(files,tgt))
    out, err1 = _run_command("/usr/bin/hdfs dfs -mkdir -p %s" % tgt)
    if err1:
        print(err1)
        return
    out, err2 = _run_command("/usr/bin/hdfs dfs -cp %s %s" % (files, tgt))
    if err2:
        print(err2)


def _hdfs_to_s3(files, tgt):
    out, err = _run_command("/usr/bin/hadoop distcp %s %s" % (files, tgt), True)
    if err:
        print(err)


def _local_to_hdfs(src, tgt):
    if tgt[0] != '/':
        tgt = '/' + tgt
    out, err1 = _run_command("/usr/bin/hdfs dfs -mkdir -p %s" % tgt)
    if err1:
        print(err1)
        return
    out, err2 = _run_command("/usr/bin/hdfs dfs -cp %s %s" % ("file://" + os.path.join(src, '*'),
                                                              tgt))
    if err2:
        print(err2)


class S3Helper:
    """A helper function to access S3 files"""
    def __init__(self):
        self.conn = None
        self.bucket_name = None
        self.bucket = None

    @staticmethod
    def help():
        print('''
        s3helper is a helper object to move files and directory between
        local filesystem, AWS S3 and local HDFS.

        Usage:

        1. Open a S3 bucket under your account
            s3helper.open_bucket(<bucket_name>)
        2. List all files under the opened S3 bucket
            s3helper.ls() or s3helper.ls_s3()
        Or optionally,
            s3helper.ls(<file_path>) or s3helper.ls_s3(<file_path>)
        3. List all files on HDFS
            s3helper.ls_hdfs()
        Or optionally,
            s3helper.ls_hdfs(<file_path>)
        where <file_path> is an absolute path in the opened S3 bucket.

        Now you can access your S3 files.

        1. Transfer files between S3 and HDFS
          a. To download all S3 files under a directory to HDFS, please call
                s3helper.s3_to_hdfs(<s3_directory_path>, <HDFS_directory_path>)
          b. To upload a directory on HDFS to S3, please call
                s3helper.hdfs_to_s3(<HDFS_directory_path>, <s3_directory_path>)

        2. Transfer files between S3 and local filesystem (not HDFS)
          a. To download one single S3 file to local filesystem, please call
                s3helper.s3_to_local(<s3_file_path>, <local_file_path>)
          b. To upload a file on local filesystem to S3, please call
                s3helper.local_to_s3(<local_file_path>, <s3_directory_path>)

        3. Transfer files between local filesystem and HDFS
          a. To upload a directory on local filesystem to HDFS, please call
                s3helper.local_to_hdfs(<local_dir_path>, <HDFS_dir_path>)

        4. Get S3 file paths without data transfer
          a. To get the URLs of S3 files under a directory, please call
                s3helper.get_path(<s3_directory_path>)
          Note this method do nothing on your local HDFS.
        ''')

    def open_bucket(self, bucket_name):
        """Open a S3 bucket.

            Args:
                bucket_name
            Returns:
                None
        """
        if bucket_name.startswith('s3n://') or bucket_name.startswith('s3://'):
            raise ValueError('bucket_name must NOT contain any prefix '
                             '(e.g. s3:// or s3n://)')

        while bucket_name[-1] == '/':
            bucket_name = bucket_name[:-1]
        self.bucket_name = bucket_name
        self.conn = S3Connection(host="s3.amazonaws.com")
        try:
            self.bucket = self.conn.get_bucket(self.bucket_name)
        except S3ResponseError as e:
            print('Open S3 bucket "%s" failed.\nError code %d: '
                  % (bucket_name, e.status) + e.reason)

    def ls(self, path=''):
        """same as ls_s3"""
        return self.ls_s3(path)

    def ls_s3(self, path=''):
        """List all files in `path` on S3.

            Args:
                path
            Returns:
                an array of files in `path`
        """
        if not self.bucket:
            raise Exception('No bucket is opened. '
                            'Please use open_bucket method first.')

        path = path.strip()
        if len(path) and path[0] == '/':
            path = path[1:]
        files = self.bucket.list(prefix=path)

        if path == '':
            k = 1
        else:
            k = len(path.split('/')) + 1
        return sorted(list(set(
            ['/'.join(t.key.split('/')[:k]) for t in files])))

    @staticmethod
    def ls_hdfs(path='/'):
        """List all files in `path` on HDFS."""
        if not path or path[0] != '/':
            path = '/' + path
        return _list_hdfs(path)

    def get_path(self, path=''):
        """Get paths of all files in `path` with s3 prefix,
           which can be passed to Spark.

            Args:
                path
            Returns:
                an array of file paths with s3 prefix
        """
        if not self.bucket:
            raise Exception('no bucket is opened.')

        path = path.strip()
        if len(path) and path[0] == '/':
            path = path[1:]
        files = self.bucket.list(prefix=path)
        prefix = "s3n://%s/" % self.bucket_name
        return [prefix + t.key for t in files]

    def s3_to_hdfs(self, src, tgt):
        """Load all files in `src` to the directory `tgt` in HDFS.

            Args:
                src, tgt
            Returns:
                an array of file paths in HDFS
        """
        if not self.bucket:
            raise Exception('no bucket is opened.')

        src, tgt = src.strip(), tgt.strip()
        print('s3_to_hdfs: src= "%s", tgt="%s"'%(src,tgt))
        if len(src) and src[0] == '/':
            src = src[1:]
        if tgt == '' or (tgt[0] != '/' and not tgt.startswith('hdfs://')):
            tgt = '/' + tgt
        if tgt[-1] != '/':
            tgt = tgt + '/'
        print("s3_to_hdfs, self.bucket.list(prefix=src), src=",src)
        files = self.bucket.list(prefix=src)
        print('s3_to_hdfs, src=',src)
        prefix = "s3n://%s/" % self.bucket_name
        print('files=[%s]'%(' '.join([prefix + t.key for t in files])))
        _s3_to_hdfs(' '.join([prefix + t.key for t in files]), tgt)
        self.ls_hdfs(tgt)

    def hdfs_to_s3(self, src, tgt):
        """Upload a directory `src` on HDFS to a directory `tgt` on S3.

           Args:
                src, tgt
           Returns:
                file list of the `tgt` directory on S3 after uploading
        """
        if not self.bucket:
            raise Exception('no bucket is opened. '
                            'See help() method for more info')

        src, tgt = src.strip(), tgt.strip()
        if src == '' or (src[0] != '/' and not src.startswith('hdfs://')):
            src = '/' + src
        if src[-1] != '/':
            src = src + '/'
        if len(tgt) and tgt[0] == '/':
            tgt = tgt[1:]
        tgt = "s3n://%s/" % self.bucket_name + tgt
        print("*NOTE*\n"
              "This method will create a MapReudce job to upload the content "
              "in HDFS to S3. The process may take a while.\n\n")
        _hdfs_to_s3(src, tgt)

    def local_to_s3(self, filename, tgt):
        """Save a local file `filename` to the directory `tgt` on S3.

            Args:
                filename, tgt
            Returns:
                None
        """
        if not self.bucket:
            raise Exception('no bucket is opened.')
        if not os.path.exists(filename):
            raise Exception("File does not exist.")
        if os.path.isdir(filename):
            raise Exception(
                "Transfer between S3 and local filesystem "
                "does not support directory.")

        tgt = tgt.strip()
        if len(tgt) and tgt[0] == '/':
            tgt = tgt[1:]
        if not tgt:
            tgt = filename.rsplit('/', 1)[-1]
        k = self.bucket.new_key(tgt)
        k.set_contents_from_filename(filename)

    def s3_to_local(self, src, tgt):
        """Download the remote file `key_name` on S3 to local.

            Args:
                src, tgt
            Returns:
                None
        """
        if not self.bucket:
            raise Exception('no bucket is opened.')

        key = src.strip()
        if key[0] == '/':
            key = key[1:]
        k = self.bucket.get_key(key)
        if not k:
            raise Exception(
                "File " + src + " doesn't exist.\n"
                "Note that the transfer between S3 and local filesystem "
                "do not support directory.")
        k.get_contents_to_filename(tgt)

    @staticmethod
    def local_to_hdfs(src, tgt):
        """Upload local directory to HDFS.

           Args:
               src - path to the local directory,
               tgt - path to the HDFS directory
           Returns:
               None
        """
        if src[0] != '/' or tgt[0] != '/':
            raise Exception("The directory path cannot be an relative path.")
        _local_to_hdfs(src, tgt)


s3helper = S3Helper()


Writing /home/hadoop/s3helper2.py


In [87]:
s3helper.s3_to_hdfs('/Weather/%s'%file, '/%s'%file)

Exception: no bucket is opened.

In [88]:
# check what is in the S3 bucket
print(s3helper.ls_s3())  # By default, list all files in the root directory of the bucket
print(s3helper.ls_s3('Weather'))

Exception: No bucket is opened. Please use open_bucket method first.

In [89]:
# check what is on HDFS
!hadoop fs -du  /weather.parquet

0         /weather.parquet/_SUCCESS
40670401  /weather.parquet/part-00000-6cb19187-62a0-42ad-9516-e03e05ea0c40-c000.snappy.parquet
40307528  /weather.parquet/part-00001-6cb19187-62a0-42ad-9516-e03e05ea0c40-c000.snappy.parquet
40016618  /weather.parquet/part-00002-6cb19187-62a0-42ad-9516-e03e05ea0c40-c000.snappy.parquet
40377232  /weather.parquet/part-00003-6cb19187-62a0-42ad-9516-e03e05ea0c40-c000.snappy.parquet
40119938  /weather.parquet/part-00004-6cb19187-62a0-42ad-9516-e03e05ea0c40-c000.snappy.parquet
40278884  /weather.parquet/part-00005-6cb19187-62a0-42ad-9516-e03e05ea0c40-c000.snappy.parquet
40078149  /weather.parquet/part-00006-6cb19187-62a0-42ad-9516-e03e05ea0c40-c000.snappy.parquet
40838598  /weather.parquet/part-00007-6cb19187-62a0-42ad-9516-e03e05ea0c40-c000.snappy.parquet
40257133  /weather.parquet/part-00008-6cb19187-62a0-42ad-9516-e03e05ea0c40-c000.snappy.parquet
40970219  /weather.parquet/part-00009-6cb19187-62a0-42ad-9516-e03e05ea0c40-c000.snappy.parquet
39737699  /wea

In [91]:
# Start SQLContext and SparkSession
from pyspark import SparkContext
from pyspark.sql import SQLContext,Row,Column
from pyspark.sql import SparkSession
sqlContext = SQLContext(sc)
#spark=SparkSession(sc)

In [92]:
measurements=['TMAX','TMIN','TOBS','SNOW','SNWD','PRCP']

In [ ]:
#read main data file
US_Weather_parquet='/US_Weather_cleaned_labeled.parquet/'
Query="SELECT * from parquet.`%s`"%US_Weather_parquet
print Query

In [ ]:
t=time()
df = sqlContext.sql(Query).cache()
print df.count()
print 'establishing SQL query took',time()-t,'seconds'

In [ ]:
t=time()
print df.count()
print 'Running SQL query took',time()-t,'seconds'

In [ ]:
from pyspark.sql.functions import *
Query='SELECT * from parquet.`/Weather_Stations.parquet`'
Station_TBL=sqlContext.sql(Query).cache()
Station_TBL.select(col('ID').alias('station'),'longitude','latitude').show(5)
#data.select(col("Name").alias("name"), col("askdaosdka").alias("age"))

In [ ]:
import numpy as np
row,=df.take(1)
unpackArray(row['vector'],data_type=np.float16)

In [ ]:
import numpy as np
t=time()

N=sc.defaultParallelism
print 'Number of executors=',N
#rdd0=df....cache().repartition(N)

# Number of executors= 80
# took 0.0209968090057 seconds

In [ ]:
t=time()
Stations=df.groupBy(df.station).count()
print 'took',time()-t,'seconds'

# took 1.15811896324 seconds
Stations.count()

In [ ]:
Stations.show(5)

In [ ]:
Stations2=Stations.join(Station_TBL.select(col('ID').alias('station'),'longitude','latitude','state'),on='station',how='left_outer')
Stations.count(), Stations2.count()

In [ ]:
Stations2.show(5)

In [ ]:
St_pandas=Stations2.toPandas()
tmp=St_pandas.sort_values('longitude')
print tmp.shape
print tmp.iloc[0,:]['longitude']
print tmp['longitude'][34738]
tmp.head()
#tmp.tail()


In [ ]:
def bisect(df,field):
    tmp=df.sort_values(field)
    l=tmp.shape[0]
    Min=tmp.iloc[0,:][field]
    Max=tmp.iloc[l-1,:][field]
    cumul=np.array(np.cumsum(tmp['count']))
    cumul=cumul/(cumul[-1]+0.0)
    midpoint=binary_search(cumul,0.5)
    #print cumul[midpoint-1:midpoint+1]
    Median=(tmp.iloc[midpoint-1,:][field]+tmp.iloc[midpoint,:][field])/2.
    return Min,Median,Max

In [ ]:
print bisect(St_pandas,'longitude')
print bisect(St_pandas,'latitude')

### Build a KD-tree.

In [ ]:
fields=['longitude','latitude']
field_i=0
def build_tree(DF,field_i,level,index):
    field=fields[field_i]
    Weight=DF['count'].sum()
    if level>0:
        min,thr,max=bisect(DF,field)
        sub1=DF[DF[field]>=thr]
        T1=build_tree(sub1,1-field_i,level-1,index+'B')
        sub2=DF[DF[field]<thr]
        T2=build_tree(sub2,1-field_i,level-1,index+'S')
        return {'min':min,
                'max':max,
                'level':level,
                'index':index,
                'field':field,
                'thr':thr,
                'Weight':Weight,
                'larger':T1,
                'smaller':T2
               }
    else:
        return Weight

In [ ]:
tree=build_tree(St_pandas,0,9,'')

In [ ]:
def calc_part(longitude, latitude,tree):
    """ Compute the part index for a given longitude and lattitude"""
    coords=[longitude,latitude]
    if tree['field']=='longitude':
        field_i=0
    else:
        field_i=1
    return find_recursive(tree,coords,field_i)

def find_recursive(tree,coords,field_i):
    if type(tree)!= dict:
        return None
    cond=coords[field_i]>tree['thr']
    if cond:
        R=find_recursive(tree['larger'],coords,1-field_i)
    else:
        R=find_recursive(tree['smaller'],coords,1-field_i)
    if R is None:
        return tree['index']
    else:
        return R

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

label_udf = udf(lambda longit,lat:calc_part(longit,lat,tree),StringType())
df_withlabel=df.withColumn("label", label_udf(df.longitude,df.latitude))
df_withlabel.cache().count()

In [ ]:
label_counts=df_withlabel.groupBy(df_withlabel.label).count().collect()

In [ ]:
Labels=[r['label'] for r in label_counts]
len(Labels)

In [ ]:
%cd /home/ec2-user/Data
!ls -l
s3helper.local_to_s3('/home/ec2-user/Data/US_Weather_SSBBSBBS.csv.gz','/Weather/small/US_Weather_SSBBSBBS.csv.gz')
!rm *

In [ ]:

from pyspark.sql import SQLContext,Row,Column
from pyspark import SparkContext
from pyspark.sql import SparkSession

sqlContext = SQLContext(sc)
spark=SparkSession(sc)
print Query

In [ ]:
t=time()
df_withlabel = sqlContext.sql(Query).cache()
print df_withlabel.count()
print 'took',time()-t,'seconds'

In [ ]:
bucket='mas-dse-open'
s3helper.open_bucket(bucket)

In [ ]:
Labels_list=df_withlabel.select('label').collect()
Labels_list[:5]

In [ ]:
import pickle
for label in Labels:
    single_label=df_withlabel.filter(df_withlabel['label']==label).collect()
    print label,len(single_label),'  ',
    #print single_label[0]
    filename='/home/ec2-user/Data/US_Weather_%s.pickle'%label
    pickle.dump(single_label,open(filename,'wb'),protocol=2)
    !gzip $filename
    !ls -l $filename*
    filename='US_Weather_%s.pickle.gz'%label
    s3helper.local_to_s3('/home/ec2-user/Data/'+filename,'/Weather/small/'+filename)

In [ ]:
t=time()
#df_withlabel.write.save('hdfs:///US_Weather_cleaned_labeled.parquet')
#print 'HDFS:\n',s3helper.ls_hdfs()
files=s3helper.hdfs_to_s3("/US_Weather_cleaned_labeled.parquet", "/Weather/US_Weather_cleaned_labeled.parquet")
print 'S3:\n',s3helper.ls_s3('Weather')
print 'took',time()-t

In [ ]:
13000*4*365./1000000.

In [ ]:
# Compute the overall distribution of values and the distribution of the number of nan per year
def find_percentiles(SortedVals,percentile):
  L=len(SortedVals)/percentile
  return SortedVals[L],SortedVals[-L]
  
def computeOverAllDist(rdd0):
  UnDef=np.array(rdd0.map(lambda row:sum(np.isnan(row))).sample(False,0.01).collect())
  flat=rdd0.flatMap(lambda v:list(v)).filter(lambda x: not np.isnan(x)).cache()
  count,S1,S2=flat.map(lambda x: np.float64([1,x,x**2]))\
                  .reduce(lambda x,y: x+y)
  mean=S1/count
  std=np.sqrt(S2/count-mean**2)
  Vals=flat.sample(False,0.0001).collect()
  SortedVals=np.array(sorted(Vals))
  low100,high100=find_percentiles(SortedVals,100)
  low1000,high1000=find_percentiles(SortedVals,1000)
  return {'UnDef':UnDef,\
          'mean':mean,\
          'std':std,\
          'SortedVals':SortedVals,\
          'low100':low100,\
          'high100':high100,\
          'low1000':low100,\
          'high1000':high1000
          }

In [ ]:
def repack_array(row):
    arr=(np.array([np.float16(row[str(i)]) for i in range(1,366)]))
    #trim extremes
    trimmed=np.array([x if (x>low1000-1) and (x<high1000+1) else np.nan for x in arr],dtype=np.float16)
    #count nan
    count=int(np.sum(np.isnan(trimmed)))
    #return row with 365 entries replaced with a bytestream representing a bytearray.

    New={}
    keep_keys=['longitude', 'station', 'latitude', 'year', 'elevation', 'measurement']
    for k in keep_keys:
        New[k]=row[k]
    New['vector']=packArray(trimmed)
    New['undefs']=count
    return Row(**New)


In [ ]:
from numpy import linalg as LA

STAT={}  # dictionary storing the statistics for each measurement
df={}
clean_df={}

for meas in measurements:
    t=time()
    Query="SELECT * FROM parquet.`%s`\n\tWHERE measurement = '%s'"%(US_Weather_parquet,meas)
    print Query
    df[meas] = sqlContext.sql(Query)
    print 'size of dataframe for ',meas,'before filtering =',df[meas].count()
    rdd0=df[meas].rdd.map(lambda row:(row['station'],((row['measurement'],row['year']),np.array([np.float64(row[str(i)]) for i in range(1,366)])))).cache()

    rdd1=rdd0.sample(False,1)\
           .map(lambda (key,val): val[1])\
           .cache()\
           .repartition(N)

    STAT[meas]=computeOverAllDist(rdd1)   # Compute the statistics 
    low1000 = STAT[meas]['low1000']  # unpack the extreme values statistics
    high1000 = STAT[meas]['high1000']

    Cleaned=df[meas].rdd.map(lambda row:repack_array(row))
    
    clean_df[meas]=spark.createDataFrame(Cleaned).filter("undefs<50")

    print 'size of dataframe for ',meas,'after filtering =',clean_df[meas].count()
    print meas,'took',time()-t,'seconds'



In [ ]:
import sys
sys.path.append('./lib')
from numpy_pack import packArray,unpackArray



In [ ]:
t=time()
df_all = clean_df['TMIN']
s=clean_df['TMIN'].count()
print 'TMIN',s
for key in ['TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP']:
    df_all=df_all.union(clean_df[key])
    s1= clean_df[key].count()
    s+= s1
    print key,s1,s
print 'total size=',df_all.cache().count()
print 'took',time()-t,'seconds'

In [ ]:
t=time()
#df_all.write.save("hdfs:///US_Weather_Cleaned.parquet")
print 'HDFS:\n',s3helper.ls_hdfs()
s3helper.hdfs_to_s3("/US_Weather_Cleaned.parquet", "/Weather/US_Weather_Cleaned.parquet")
print 'S3:\n',s3helper.ls_s3('Weather')
print 'took',time()-t
# took took 525.435763121